<img src="./Img/banner-fa-49.png">

Учебные материалы дисциплины "Программирование на языках Python и SQL" предназначены для семинарских занятий со студентами II курса Финансового университа при Правительстве Российской Федерации.

Автор: Смирнов Михаил Викторович, доцент Департамента анализа данных и машинного обучения Финансового университета при Правительстве Российской Федерации. mvsmirnov@fa.ru

Москва - 2021

При подготовке материалов учебных занятий использовались источники
- Essential SQLAlchemy: Mapping Python to Databases 2nd Edition. Jason Myers, Rick Copeland. O'Reilly Media, Inc. 2015.
- Астахова И.Ф., Мельников В.М., Толстобров А.П., Фертиков В.В. СУБД: язык SQL в примерах и задачах.—М.:ФИЗМАТЛИТ, 2009. — 168 с. — ISBN 978-5-9221-0816-4.

В ряде учебных примеров использованы данные <a href="http://insideairbnb.com/get-the-data.html">Inside Airbnb</a>

<a id=T_3></a>
# Тема 3. SQLAlchemy ORM

<a id=Ref></a>
# Оглавление

[Введение](#Intro)<br>
[3.1. Определение таблиц через классы ORM](#T_3_1)<br>
[3.2. Работа с данными](#T_3_2)<br>
[3.3. Запросы к данным](#T_3_3)<br>
[3.4. Исключения и транзакции](#T_3_4)<br>
[3.5. Отображение](#T_3_5)<br>

<img src="./Img/Label_02.png">

Семинар № 9

16 апреля 2021 года <br>
ПИ19-3, ПИ19-4 - 3 подгруппа<br>

17 апреля 2021 года <br>
ПИ19-2, ПИ19-3, ПИ19-4 - 2 подгруппа

23 апреля 2021 года <br>
ПИ19-4, ПИ19-5 - 4 подгруппа

<a id=Intro></a>
# Введение
[<= ](#Ref)||[ К оглавлению ](#Ref)||[ =>](#T_3_1)

ORM - Object-relational mapping - Объектно-реляционное отображение

SQLAlchemy ORM обеспечивает эффективный способ привязки схемы и операций базы данных к объектам данных.

В SQLAlchemy Core мы создавали контейнер метаданных, а затем объявляли объект *Table*, связанный с этими метаданными. В SQLAlchemy ORM мы будем определять класс, который наследуется от специального базового класса *declarative_base*. Этот базовый класс объединяет контейнер метаданных и средство сопоставления, которое сопоставляет наш класс с таблицей базы данных. Он также сопоставляет экземпляры класса с записями в этой таблице.

<a id=T_3_1></a>
[<= ](#Intro)||[ К оглавлению ](#Ref)||[ =>](#T_3_2)

# 3.1. Определение таблиц через классы ORM

ORM классы должны:
- Происходить от класса *declarative_base*.
- Содержать `__tablename__`, которое является именем таблицы базы данных.
- Содержать один или несколько атрибутов, которые являются объектами *Column*.
- Содержать атрибуты, составляющие первичный ключ.

Изучим требование, связанное с атрибутами. Определение столбцов в классе ORM похоже на определение столбцов в объекте *Table*, которое мы изучили в теме SQLAlchemy Core. Однако есть важное отличие. При определении столбцов в классе ORM в качестве имени столбца будет установлено имя атрибута класса, которому он назначен. Все остальное, что связано с типами данных и столбцами, применимо и здесь.

<img src="./Img/Listings_ORM_Schema.png">

<br><br>
Определим таблицу *listings* как класс ORM

In [1]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import (Table, Column, 
                        Integer, Numeric, String, Boolean,
                        ForeignKey, ForeignKeyConstraint, CheckConstraint)

from datetime import datetime
from sqlalchemy import DateTime

Base = declarative_base()

In [2]:
class Listing(Base):
    __tablename__ = 'listings'

    listing_id = Column(Integer(), primary_key = True)
    listing_name = Column(String(50), index = True, nullable = False)
    listing_url = Column(String(50))
    host_id = Column(Integer())
    neighbourhood_id = Column(Integer())
    amenities = Column(String(250))
    property_type_id = Column(Integer())
    room_type_id = Column(Integer())
    bedrooms = Column(Integer())
    beds = Column(Integer())
    price = Column('price',Numeric(7,2))
    
    __table_args__ = (
        ForeignKeyConstraint(['neighbourhood_id'],['neighbourhoods.neigh_id']),
        ForeignKeyConstraint(['property_type_id'], ['property_types.property_type_id']),
        ForeignKeyConstraint(['room_type_id'], ['room_types.room_type_id']),
        CheckConstraint('price >= 0.00', name='listing_price_positive')
    )
    

В этом примере *Base* - экземпляр класса *declarative_base()*. Затем создается дочерний класс *Listings*. Определяется имя таблицы 'listings'. Определяются атрибуты, устанавливается первичный ключ. Обратимся к свойству `__table__` класса.

In [3]:
Listing.__table__

Table('listings', MetaData(), Column('listing_id', Integer(), table=<listings>, primary_key=True, nullable=False), Column('listing_name', String(length=50), table=<listings>, nullable=False), Column('listing_url', String(length=50), table=<listings>), Column('host_id', Integer(), table=<listings>), Column('neighbourhood_id', Integer(), ForeignKey('neighbourhoods.neigh_id'), table=<listings>), Column('amenities', String(length=250), table=<listings>), Column('property_type_id', Integer(), ForeignKey('property_types.property_type_id'), table=<listings>), Column('room_type_id', Integer(), ForeignKey('room_types.room_type_id'), table=<listings>), Column('bedrooms', Integer(), table=<listings>), Column('beds', Integer(), table=<listings>), Column('price', Numeric(precision=7, scale=2), table=<listings>), schema=None)

Создадим класс для клиентов

In [4]:
from datetime import datetime
from sqlalchemy import DateTime

class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key = True)
    username = Column(String(15), nullable = False, unique = True)
    email_address = Column(String(255), nullable = False)
    phone = Column(String(20), nullable = False)
    password = Column(String(25), nullable = False)
    created_on = Column(DateTime(), default = datetime.now)
    updated_on = Column(DateTime(), default = datetime.now, onupdate=datetime.now)

Здесь мы определили несколько атрибутов, которые не могут оставаться пустыми. Требуется уникальное значение *username*. Для атрибута *updated_on* мы установили текущее время по умолчанию, если время не указано. Использование *onupdate* приведет к установке текущего времени при обновлении любого атрибута записи.

## 3.1.1. Ключи, ограничения, индексы

Ранее, в разделе *Core*, мы изучили, что ключи и ограничения могут задаваться как в составе элемента *Column()* конструктора *Table*, так и в явном виде. Например, в *line_items* атрибут *order_id* является внешним ключом, тода

`ForeignKeyConstraint(['order_id'], ['order.order_id'])`. 

В ORM также существует для этого два способа, но так как конструктор *Table* здесь не используется, то применяются свойства класса. 

```
user_id = Column(Integer(), ForeignKey('users.user_id'))
```

Для задания ограничения в явном виде в классе используется `__table_args__`

```
class SomeDataClass(Base):
    __tablename__ = 'somedatatable'
    __table_args__ = (ForeignKeyConstraint(['id'], ['other_table.id']),
                      CheckConstraint(price >= 0.00', name='unit_cost_positive'))
```
В данном примере значением `__table_args__` является кортеж.

### Задание 3.1.1.1.
Создать классы *Order, Line_item, Host, Neighbourhood, Room_type, Property_type*. Создать базу данных *Listings.db*

In [5]:
# Ваш код здесь


In [6]:
class Order(Base):

    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key = True)
    user_id = Column(Integer())
    
    __table_args__ = (ForeignKeyConstraint(['user_id'], ['users.user_id']),)

In [7]:
Order.__table__

Table('orders', MetaData(), Column('order_id', Integer(), table=<orders>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.user_id'), table=<orders>), schema=None)

In [8]:
class Line_item(Base):
    
    __tablename__ = 'line_items'
    item_id = Column(Integer(), primary_key = True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    listing_id = Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date = Column(DateTime(), nullable = False, default = datetime.now)
    item_end_date = Column('item_end_date', DateTime(), nullable = False)

In [9]:
Line_item.__table__

Table('line_items', MetaData(), Column('item_id', Integer(), table=<line_items>, primary_key=True, nullable=False), Column('order_id', Integer(), ForeignKey('orders.order_id'), table=<line_items>), Column('listing_id', Integer(), ForeignKey('listings.listing_id'), table=<line_items>), Column('item_start_date', DateTime(), table=<line_items>, nullable=False, default=ColumnDefault(<function datetime.now at 0x1112ea3a0>)), Column('item_end_date', DateTime(), table=<line_items>, nullable=False), schema=None)

In [10]:
class Host(Base):
    
    __tablename__ = 'hosts'
    host_id = Column(Integer(), primary_key = True)
    host_name = Column(String(50), nullable = False)

In [11]:
class Neighbourhood(Base):
    
    __tablename__ = 'neighbourhoods'
    neigh_id = Column(Integer(), primary_key = True)
    neigh_name = Column(String(50), nullable = False, unique = True)

In [12]:
class Room_type(Base):
    
    __tablename__ = 'room_types'
    room_type_id = Column(Integer(), primary_key = True)
    room_type_name = Column(String(50), nullable = False)

In [13]:
class Property_type(Base):
    
    __tablename__ = 'property_types'
    property_type_id = Column(Integer(), primary_key = True)
    property_type_name = Column(String(50), nullable = False)

## 3.1.2. Сохранение схемы

In [14]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///Listings.db')

Base.metadata.create_all(engine)

## 3.1.3. Связи
В *ORM* имеются некоторые различия при связывании таблиц по сравнению с *Core*. *ORM* также использует *ForeignKey* для ограничения и связывания объектов. Однако *ORM* использует директиву *relationship* чтобы предоставить свойство доступа к связанному объекту. Это добавляет некоторые накладные расходы при использовании *ORM*; однако плюсы перевешивают недостатки. В примере показано, как определить связи с помощью методов *relationship* и *backref*.
```
from sqlalchemy.orm import relationship, backref

class Orders(Base):
    __tablename__='orders'
    order_id=Column(Integer(), primary_key=True)
    user_id=Column(Integer(), ForeignKey('users.user_id'))

    User=relationship('Users', backref=backref('orders', order_by=order_id))
```
Таким образом, в классе *Orders*, устанавливается отношение «один ко многим» с классом *Users*. Мы можем связать пользователя с его заказом, обратившись к свойству *user*. Это отношение также устанавливает свойство *orders* в классе *Users* через аргумент ключевого слова *backref*, которое упорядочивается по *order_id*. Директиве *relationship* требуется целевой класс для отношения, и она может дополнительно включать обратное отношение для целевого класса. *SQLAlchemy* знает, как сопоставить заданный нами *ForeignKey* с классом, который мы определили в отношении. В этом примере команда `ForeignKey(users.user_id)` сопоставляется с классом *User* через атрибут `__tablename__` пользователей и формирует связь. В строке 
```
User=relationship('Users', backref=backref('orders', order_by=order_id))
```
устанавливается связь *один ко многим*.

Также возможно установить взаимно-однозначное отношение *один к одному*. В следующем примере класс *Line_items* имеет взаимно-однозначное отношение с классом *Listings*. Аргумент ключевого слова `uselist = False` определяет его как взаимно однозначное отношение. Здесь используется более простая обратная ссылка, поскольку нам не нужно контролировать порядок.
```
class Line_items(Base):
    
    __tablename__='line_items'
    item_id=Column(Integer(), primary_key=True)
    order_id=Column(Integer(), ForeignKey('orders.order_id'))
    listing_id=Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date=Column(DateTime(), nullable=False, default=datetime.now)
    item_end_date=Column('item_end_date', DateTime(), nullable=False)
    
    Order=relationship("Orders", backref=backref('line_items', order_by=line_item_id))
    Listing=relationship("Listings", uselist=False))
```

<a id=T_3_2></a>
[<= ](#T_3_1)||[ К оглавлению ](#Ref)||[ =>](#T_3_3)
# 3.2. Работа с данными


## 3.2.1. Сеанс

Сеанс - это способ взаимодействия *ORM SQLAlchemy* с базой данных. Она "обертывает" соединение с базой данных через механизм и предоставляет карту идентификации для объектов, которые вы загружаете через сеанс или связываете с сеансом. Карта идентификации - это структура данных, подобная кешу, которая содержит уникальный список объектов, определяемый таблицей объекта и первичным ключом. Сеанс также "обертывает" транзакцию, и эта транзакция будет открыта до тех пор, пока сеанс не будет зафиксирован или не пройзойдет откат, что очень похоже на процесс, описанный в теме *Core*.

Для нового сеанса SQLAlchemy предоставляет класс *sessionmaker*, чтобы гарантировать, что сеансы могут быть созданы с одинаковыми параметрами во всем приложении. SQLAlchemy делает это путем создания класса сеанса (Session), который настроен в соответствии с аргументами, переданными в класс *sessionmaker*, который следует использовать только один раз в глобальной области действия приложения и рассматривать как параметр конфигурации. Создадим новый сеанс, связанный с базой данных SQLite в памяти:

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker # 1 

engine = create_engine('sqlite:///:memory:') # 2

Session = sessionmaker(bind=engine) # 3

session = Session() # 4

1. Импорт модуля создания сеанса *sessionmaker*
2. База данных SQLite в памяти
3. Определение класса сеанса с привязкой к механизму.
4. Создание сеанса.

Теперь у нас есть сеанс, который мы можем использовать для взаимодействия с базой данных. Определим классы таблиц базы данных. Дополнительно добавим методы `__repr__`, чтобы упростить просмотр и воссоздание экземпляров объектов.

In [4]:
from datetime import datetime

from sqlalchemy import (Table, Column, 
                        Integer, Numeric, String, Boolean, DateTime,
                        ForeignKey, ForeignKeyConstraint, CheckConstraint)

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

Base = declarative_base()


class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key = True)
    username = Column(String(15), nullable = False, unique = True)
    email_address = Column(String(255), nullable = False)
    phone = Column(String(20), nullable = False)
    password = Column(String(25), nullable = False)
    created_on = Column(DateTime(), default = datetime.now)
    updated_on = Column(DateTime(), default = datetime.now, onupdate=datetime.now)

    def __repr__(self):
        return "User(username='{self.username}', " \
                     "email_address='{self.email_address}', " \
                     "phone='{self.phone}', " \
                     "password='{self.password}')".format(self=self)

    
class Order(Base):

    __tablename__ = 'orders'
    order_id = Column(Integer(), primary_key = True)
    user_id = Column(Integer())
    
    __table_args__ = (ForeignKeyConstraint(['user_id'], ['users.user_id']),)
    
    User=relationship("User", backref=backref('orders', order_by=order_id))
    
    def __repr__(self):
        return "Order(user_id='{self.user_id}')".format(self=self)
    

class Line_item(Base):
    
    __tablename__ = 'line_items'
    item_id = Column(Integer(), primary_key = True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    listing_id = Column(Integer(), ForeignKey('listings.listing_id'))
    item_start_date = Column(DateTime(), nullable = False, default = datetime.now)
    item_end_date = Column('item_end_date', DateTime(), nullable = False)
    
    Order=relationship("Order", backref=backref('line_items', order_by=item_id))
    Listing=relationship("Listing", uselist=False)
    
    def __repr__(self):
        return "Line_item(order_id='{self.order_id}', " \
                        "listing_id='{self.listing_id}', " \
                        "item_start_date='{self.item_start_date}', " \
                        "item_end_date='{self.item_end_date}')".format(self=self)


class Host(Base):
    
    __tablename__ = 'hosts'
    host_id = Column(Integer(), primary_key = True)
    host_name = Column(String(50), nullable = False)
    def __repr__(self):
        return "Host(host_id='{self.host_name}')".format(self=self)
    
    
class Neighbourhood(Base):
    
    __tablename__ = 'neighbourhoods'
    neigh_id = Column(Integer(), primary_key = True)
    neigh_name = Column(String(50), nullable = False, unique = True)
    def __repr__(self):
        return "Neighbourhood(neigh_name='{self.neigh_name}')".format(self=self)
    
    
class Room_type(Base):
    
    __tablename__ = 'room_types'
    room_type_id = Column(Integer(), primary_key = True)
    room_type_name = Column(String(50), nullable = False)
    def __repr__(self):
        return "Room_type(room_type_name='{self.room_type_name}')".format(self=self)

### Задание 3.2.1.1.

Создайте класс *Property_type* для справочника типов собственности *property_types*

In [5]:
# Ваш код здесь


In [6]:
class Property_type(Base):
    
    __tablename__ = 'property_types'
    property_type_id = Column(Integer(), primary_key = True)
    property_type_name = Column(String(50), nullable = False)
    def __repr__(self):
        return "Property_type(property_type_name='{self.property_type_name}')".format(self=self)

### Задание 3.2.1.2.
Создайте класс *Listing* с отношениями к классам *Host, Neighbourhood, Room_type, Property_type, Line_item* и методами `__repr__`

In [7]:
# Ваш код здесь


In [8]:
class Listing(Base):
    __tablename__ = 'listings'

    listing_id = Column(Integer(), primary_key = True)
    listing_name = Column(String(50), index = True, nullable = False)
    listing_url = Column(String(50))
    host_id = Column(Integer())
    neighbourhood_id = Column(Integer())
    amenities = Column(String(250))
    property_type_id = Column(Integer())
    room_type_id = Column(Integer())
    bedrooms = Column(Integer())
    beds = Column(Integer())
    price = Column('price',Numeric(7,2))
    
    __table_args__ = (
        ForeignKeyConstraint(['neighbourhood_id'],['neighbourhoods.neigh_id']),
        ForeignKeyConstraint(['property_type_id'], ['property_types.property_type_id']),
        ForeignKeyConstraint(['room_type_id'], ['room_types.room_type_id']),
        ForeignKeyConstraint(['host_id'], ['hosts.host_id']),
        CheckConstraint('price >= 0.00', name='listing_price_positive')
    )
    
    Host=relationship('Host', backref=backref('listings', order_by=listing_id))
    Neighbourhood=relationship('Neighbourhood', backref=backref('listings', order_by=listing_id))
    Property_type=relationship('Property_type', backref=backref('listings', order_by=listing_id))
    Room_type=relationship('Room_type', backref=backref('listings', order_by=listing_id))

    
    def __repr__(self):
        return "Listing(listing_name='{self.listing_name}', " \
                       "listing_url='{self.listing_url}', " \
                       "amenities='{self.amenities}', " \
                       "bedrooms='{self.bedrooms}', " \
                       "price='{self.price}')".format(self=self)

In [9]:
Base.metadata.create_all(engine)

## 3.2.2. Вставка данных

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('./Data/ListingsAm.csv', sep=";")
df.head(1)

,id,listing_url,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,property_type,room_type,bathrooms_text,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,Random
0,20168,https://www.airbnb.com/rooms/20168,Studio with private bathroom in the centre 1,59484,Alexander,f,Centrum-Oost,Private room in townhouse,Private room,1 private bath,...,2020-04-09,89,10.0,10.0,10.0,10.0,10.0,9.0,2.58,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3436 entries, 0 to 3435
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           3436 non-null   int64  
 1   listing_url                  3436 non-null   object 
 2   name                         3433 non-null   object 
 3   host_id                      3436 non-null   int64  
 4   host_name                    3436 non-null   object 
 5   host_is_superhost            3436 non-null   object 
 6   neighbourhood_cleansed       3436 non-null   object 
 7   property_type                3436 non-null   object 
 8   room_type                    3436 non-null   object 
 9   bathrooms_text               3433 non-null   object 
 10  bedrooms                     3286 non-null   float64
 11  beds                         3435 non-null   float64
 12  amenities                    3436 non-null   object 
 13  price             

Подготовим справочник районов

In [13]:
neigh_df = df["neighbourhood_cleansed"].value_counts().sort_index().reset_index()
neigh_df.index=range(1, len(neigh_df)+1)
neigh_dict = neigh_df["index"].to_dict()
neigh_dict

{1: 'Bos en Lommer',
 2: 'Buitenveldert - Zuidas',
 3: 'Centrum-Oost',
 4: 'Centrum-West',
 5: 'De Aker - Nieuw Sloten',
 6: 'De Baarsjes - Oud-West',
 7: 'De Pijp - Rivierenbuurt',
 8: 'Geuzenveld - Slotermeer',
 9: 'IJburg - Zeeburgereiland',
 10: 'Noord-Oost',
 11: 'Noord-West',
 12: 'Oostelijk Havengebied - Indische Buurt',
 13: 'Osdorp',
 14: 'Oud-Noord',
 15: 'Oud-Oost',
 16: 'Slotervaart',
 17: 'Watergraafsmeer',
 18: 'Westerpark',
 19: 'Zuid'}

In [14]:
for key, value in neigh_dict.items():
    w = Neighbourhood(neigh_id = key, neigh_name = value)
    session.add(w)
session.commit()

In [15]:
print(w.neigh_id, w.neigh_name)

19 Zuid


### Задание 3.2.2.1.
Создайте справочники владельцев недвижимости, типов комнат и типов собственности. При создании справочника владельцев в качестве значения первичного ключа укажите значение индекса владельца из *ListingsAm.csv*

Чтобы добавить запись, содержащую значения нескольких столбцов, например первичного ключа и названия, можно в одном выражении указать несколько параметров класса таблицы:

```
w=ClassName(Column_1=value_1, Column_2=value_2, ... Column_n=value_n)
sesseion.add(w)
```

In [16]:
# Ваш код здесь


In [17]:
H=df[['host_id','host_name']].groupby(["host_id", "host_name"])['host_id'].count()
H.head()

host_id  host_name      
47517    Geert Alexander    1
59484    Alexander          1
61977    Marjolein          1
81046    Fabienne           1
92194    Taco & Iwanna      1
Name: host_id, dtype: int64

In [18]:
item=H.index[1]
item[1]

'Alexander'

In [19]:
for item in H.index:
    w = Host(host_id=item[0], host_name=item[1])
    session.add(w)

session.commit()
w.host_id, w.host_name

(331113200, 'Jennifer')

Проверка, что записано в таблицу *hosts*, сколько записей в таблице *hosts*.

In [20]:
Host.__table__

Table('hosts', MetaData(), Column('host_id', Integer(), table=<hosts>, primary_key=True, nullable=False), Column('host_name', String(length=50), table=<hosts>, nullable=False), schema=None)

In [21]:
from sqlalchemy import select
connection = engine.connect()
sel = select([Host])
sel = sel.limit(3)
rp = connection.execute(sel)
rp.fetchall()

[(47517, 'Geert Alexander'), (59484, 'Alexander'), (61977, 'Marjolein')]

In [22]:
from sqlalchemy.sql import func
s=select([func.count(Host.__table__)])
print(str(s))
rp = connection.execute(s)
rp.scalar()

ArgumentError: SQL expression element expected, got Table('hosts', MetaData(), Column('host_id', Integer(), table=<hosts>, primary_key=True, nullable=False), Column('host_name', String(length=50), table=<hosts>, nullable=False), schema=None). To create a column expression from a FROM clause row as a whole, use the .table_valued() method.

Наполнение справочника типов комнат

In [23]:
room_type_df = df["room_type"].value_counts().sort_index().reset_index()
room_type_df.index = range(1, len(room_type_df)+1)
room_type_dict=room_type_df["index"].to_dict()

for value in room_type_dict.values():
    w = Room_type(room_type_name = value)
    session.add(w)

session.commit()
w.room_type_id, w.room_type_name

(4, 'Shared room')

Наполнение справочников типов собственности

In [24]:
property_type_df = df["property_type"].value_counts().sort_index().reset_index()
property_type_df.index = range(1, len(property_type_df)+1)
property_type_dict = property_type_df["index"].to_dict()

for value in property_type_dict.values():
    w = Property_type(property_type_name = value)
    session.add(w)

session.commit()
w.property_type_id, w.property_type_name

(42, 'Tiny house')

### Задание 3.2.2.2.

Наполните данными таблицу *listings*

In [25]:
# Ваш код здесь

In [26]:
df.head(1)

,id,listing_url,name,host_id,host_name,host_is_superhost,neighbourhood_cleansed,property_type,room_type,bathrooms_text,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,Random
0,20168,https://www.airbnb.com/rooms/20168,Studio with private bathroom in the centre 1,59484,Alexander,f,Centrum-Oost,Private room in townhouse,Private room,1 private bath,...,2020-04-09,89,10.0,10.0,10.0,10.0,10.0,9.0,2.58,0


В процессе работы выдается предупреждение о том, что тип данных "десятичное число" не поддерживается напрямую. Чтобы отключить такие сообщения, выполним команду
```
import warnings; warnings.simplefilter('ignore')
```

In [27]:
import warnings; warnings.simplefilter('ignore')

neigh_dict_reversed = {}
for key, value in neigh_dict.items():
    neigh_dict_reversed[value]=key

property_type_dict_reversed = {}
for key, value in property_type_dict.items():
    property_type_dict_reversed[value] = key

room_type_dict_reversed = {}
for key, value in room_type_dict.items():
    room_type_dict_reversed[value] = key
    
for row in df.index[::100]:
    w = Listing(
        listing_name = df.loc[row, 'name'],
        listing_url = df.loc[row, 'listing_url'],
        host_id = int(df.loc[row, 'host_id']),
        neighbourhood_id = neigh_dict_reversed[df.loc[row, 'neighbourhood_cleansed']],
        amenities = df.loc[row, 'amenities'],
        property_type_id = property_type_dict_reversed[df.loc[row, 'property_type']],
        room_type_id = room_type_dict_reversed[df.loc[row, 'room_type']],
        bedrooms = df.loc[row, 'bedrooms'],
        beds = df.loc[row, 'beds'],
        price = df.loc[row, 'price']
    )
    
    session.add(w)

session.commit()
print(w.listing_id, w.listing_url, w.listing_name, 
      w.host_id, w.neighbourhood_id, w.property_type_id, w.room_type_id)

35 https://www.airbnb.com/rooms/43574913 Quad room / 2 double beds - Double Impact  243118788 7 36 3


<img src="./Img/Label_02.png">

Семинар

30 апреля 2021 года <br>
ПИ19-3, ПИ19-4 - 3 подгруппа<br>

30 апреля 2021 года <br>
ПИ19-4, ПИ19-5 - 4 подгруппа

15 мая 2021 года <br>
ПИ19-2, ПИ19-3, ПИ19-4 - 2 подгруппа

## 3.2.3. Запросы

### Методы *first(), all(), one().* Ограничение *limit()*.

In [28]:
session.query(Room_type).all()

[Room_type(room_type_name='Entire home/apt'),
 Room_type(room_type_name='Hotel room'),
 Room_type(room_type_name='Private room'),
 Room_type(room_type_name='Shared room')]

In [29]:
session.query(Property_type).limit(10).all()

[Property_type(property_type_name='Boat'),
 Property_type(property_type_name='Entire apartment'),
 Property_type(property_type_name='Entire bed and breakfast'),
 Property_type(property_type_name='Entire condominium'),
 Property_type(property_type_name='Entire cottage'),
 Property_type(property_type_name='Entire guest suite'),
 Property_type(property_type_name='Entire guesthouse'),
 Property_type(property_type_name='Entire house'),
 Property_type(property_type_name='Entire loft'),
 Property_type(property_type_name='Entire place')]

In [30]:
session.query(Neighbourhood.neigh_id, Neighbourhood.neigh_name).first()

(1, 'Bos en Lommer')

In [31]:
for item in session.query(Room_type):
    print(item.room_type_id, item.room_type_name)

1 Entire home/apt
2 Hotel room
3 Private room
4 Shared room


In [32]:
session.query(Listing).limit(1).one()

Listing(listing_name='Studio with private bathroom in the centre 1', listing_url='https://www.airbnb.com/rooms/20168', amenities='["Wifi", "Hot water", "Hangers", "Host greets you", "Long term stays allowed", "Carbon monoxide alarm", "Fire extinguisher", "Dedicated workspace", "Paid parking off premises", "Essentials", "Bed linens", "Hair dryer", "TV", "Heating", "Refrigerator", "Free street parking", "Smoke alarm"]', bedrooms='1', price='236.00')

In [33]:
session.query(Room_type.room_type_id, Room_type).all()

[(1, Room_type(room_type_name='Entire home/apt')),
 (2, Room_type(room_type_name='Hotel room')),
 (3, Room_type(room_type_name='Private room')),
 (4, Room_type(room_type_name='Shared room'))]

## Ограничение числа строк запроса

In [34]:
session.query(
    Listing.listing_id,
    Listing.room_type_id,
    Listing.property_type_id,
    Listing.neighbourhood_id,
    Listing.host_id
)[7:11]

[(8, 1, 2, 7, 7473277),
 (9, 1, 2, 15, 2737248),
 (10, 1, 2, 7, 15239755),
 (11, 3, 17, 9, 47479242)]

## Ограничение числа строк запроса

In [35]:
session.query(
    Listing.listing_id, 
    Listing.listing_name
).order_by(Listing.listing_id).all()

[(1, 'Studio with private bathroom in the centre 1'),
 (2, 'Green oasis only 15 min from Station.'),
 (3, 'Great View Apartment in the Jordaan'),
 (4, 'Characteristic, Sunny and Colourful Apartment'),
 (5, 'COSY HOUSEBOAT IN THE CITY CENTER'),
 (6, 'Room in the centre of Amsterdam'),
 (7, 'Beautifull townhouse in the hart of Amsterdam'),
 (8, 'Bright apt. in middle of de Pijp!'),
 (9, 'Mini-Loft in trendy East'),
 (10, 'Luxury and modern apartment @Pijp'),
 (11, 'Luxurious Room For One 15 Min. From City Centre'),
 (12, "Spacious apartment in the ' pijp '"),
 (13, 'View on the canals'),
 (14, 'Luxerious and Spacious Apartment'),
 (15, 'Light and modern mini-loft close to citycenter'),
 (16, 'Light / Quiet apartment in Amsterdam city centre'),
 (17, 'Bright private rooms | Centre of Amsterdam'),
 (18, 'Bright and spacious corner apartment with parkview'),
 (19, 'Light and cosy apartment near city centre'),
 (20, '2 bedrooms in the heart of Amsterdam'),
 (21, 'Comfortable apartment next t

По убыванию

In [36]:
session.query(
    Listing.listing_id, 
    Listing.listing_name
).order_by(Listing.listing_id.desc()).all()

[(35, 'Quad room / 2 double beds - Double Impact\xa0'),
 (34, 'Quiet and spaceous apptmnt in Noord- 5min. to CS'),
 (33, 'Trendy apartment 5 min from the center f Amsterdam'),
 (32, 'Sunny two floor apartment 8 min walk from RAI'),
 (31, 'Spacious and characteristic city apartment +garden'),
 (30, 'Lovely, authentic and clean apartment.'),
 (29, 'Great private room/apartment in the center.'),
 (28, 'New beautiful design studio in Amsterdam West'),
 (27, 'Unforgetable Charming Apartment | Patio |'),
 (26, 'Quiet room/private bathroom near centre Amsterdam'),
 (25, 'Bright & beautiful apartment 10 mins from centre'),
 (24, 'Comfy, stylish & quiet apt. for 2 in De Baarsjes'),
 (23, 'Private suite next Westerpark'),
 (22, 'Second Floor Apartment'),
 (21, 'Comfortable apartment next to city centre | 2BD'),
 (20, '2 bedrooms in the heart of Amsterdam'),
 (19, 'Light and cosy apartment near city centre'),
 (18, 'Bright and spacious corner apartment with parkview'),
 (17, 'Bright private rooms

In [37]:
for item in session.query(

    Listing.listing_id,
    Listing.listing_name,
    Listing.price,
    Listing.neighbourhood_id).order_by(Listing.listing_name.desc())[:10]:
    print(item.listing_name, item.price)

View on the canals 100.00
Unforgetable Charming Apartment | Patio | 270.00
Trendy apartment 5 min from the center f Amsterdam 140.00
Sunny two floor apartment 8 min walk from RAI 450.00
Studio with private bathroom in the centre 1 236.00
Spacious apartment in the ' pijp ' 191.00
Spacious and characteristic city apartment +garden 225.00
Second Floor Apartment 700.00
Room in the centre of Amsterdam 88.00
Quiet room/private bathroom near centre Amsterdam 100.00


## Встроенные функции

In [38]:
from sqlalchemy import func

In [39]:
print(session.query(func.sum(Listing.price)).scalar())
type(session.query(func.sum(Listing.price)).scalar())

5416.00


decimal.Decimal

In [40]:
type(session.query(func.sum(Listing.price)).one())

sqlalchemy.engine.row.Row

In [41]:
print(session.query(func.sum(Listing.price)).first())

(Decimal('5416.00'),)


In [42]:
## Фильтрация 

In [43]:
from pprint import pprint
record=session.query(Listing.listing_id, Listing).filter(Listing.bedrooms==3).all()
pprint(record)

[(32, Listing(listing_name='Sunny two floor apartment 8 min walk from RAI', listing_url='https://www.airbnb.com/rooms/36303271', amenities='["Wifi", "Iron", ... (148 characters truncated) ... id kit", "Hair dryer", "TV", "Heating", "Kitchen", "Washer", "Refrigerator", "Cooking basics", "Smoke alarm", "Stove"]', bedrooms='3', price='450.00'))]


In [44]:
records = session.query(Listing.listing_id, Listing) \
    .filter(Listing.price>300) \
    .filter(Listing.bedrooms>1).all()
print(records)

[(22, Listing(listing_name='Second Floor Apartment', listing_url='https://www.airbnb.com/rooms/20953659', amenities='["Microwave", "Refrigerator", "Dishes a ... (199 characters truncated) ... n", "Dishwasher", "Paid parking off premises", "Dryer", "Essentials", "Bed linens", "Hair dryer", "Heating", "Washer"]', bedrooms='2', price='700.00')), (32, Listing(listing_name='Sunny two floor apartment 8 min walk from RAI', listing_url='https://www.airbnb.com/rooms/36303271', amenities='["Wifi", "Iron", ... (148 characters truncated) ... id kit", "Hair dryer", "TV", "Heating", "Kitchen", "Washer", "Refrigerator", "Cooking basics", "Smoke alarm", "Stove"]', bedrooms='3', price='450.00'))]


In [45]:
#Host, где имя = Александр


records = session.query(Host.host_id, Host)\
    .filter(Host.host_name.like("%Alexander%")).all()
pprint(records)

[(47517, Host(host_id='Geert Alexander')),
 (59484, Host(host_id='Alexander')),
 (8275150, Host(host_id='Alexander')),
 (26045647, Host(host_id='Alexander')),
 (72822537, Host(host_id='Alexander')),
 (207287996, Host(host_id='Alexander'))]


In [50]:

from sqlalchemy import and_, or_, not_

session.query(Listing.listing_id, Listing).filter(

    and_(
        Listing.bedrooms>1,
        Listing.price.between(175,200)
    )
).all()

[(7, Listing(listing_name='Beautifull townhouse in the hart of Amsterdam', listing_url='https://www.airbnb.com/rooms/5034152', amenities='["Stove", "Heatin ... (300 characters truncated) ...  "Kitchen", "Host greets you", "Hangers", "Washer", "Paid parking off premises", "Smoke alarm", "Dedicated workspace"]', bedrooms='2', price='175.00')),
 (25, Listing(listing_name='Bright & beautiful apartment 10 mins from centre', listing_url='https://www.airbnb.com/rooms/24439562', amenities='["Microwave", ... (262 characters truncated) ... id parking off premises", "Essentials", "Extra pillows and blankets", "Bed linens", "Hair dryer", "Heating", "Washer"]', bedrooms='2', price='200.00'))]

## Update

In [55]:
q = session.query(Listing)
w = q.filter(Listing.listing_id == 25).first()
print(w)
w.price = w.price + 1
session.commit()
print(w.price)

Listing(listing_name='Bright & beautiful apartment 10 mins from centre', listing_url='https://www.airbnb.com/rooms/24439562', amenities='["Microwave", "First aid kit", "Refrigerator", "Dishes and silverware", "Oven", "Patio or balcony", "Stove", "Wifi", "Hot water", "Hangers", "Shampoo", "Luggage dropoff allowed", "Kitchen", "Cooking basics", "Iron", "Dishwasher", "Host greets you", "Dedicated workspace", "Paid parking off premises", "Essentials", "Extra pillows and blankets", "Bed linens", "Hair dryer", "Heating", "Washer"]', bedrooms='2', price='200.00')
201.00


In [56]:
session.query(Listing).filter(Listing.listing_id == 25).first()

Listing(listing_name='Bright & beautiful apartment 10 mins from centre', listing_url='https://www.airbnb.com/rooms/24439562', amenities='["Microwave", "First aid kit", "Refrigerator", "Dishes and silverware", "Oven", "Patio or balcony", "Stove", "Wifi", "Hot water", "Hangers", "Shampoo", "Luggage dropoff allowed", "Kitchen", "Cooking basics", "Iron", "Dishwasher", "Host greets you", "Dedicated workspace", "Paid parking off premises", "Essentials", "Extra pillows and blankets", "Bed linens", "Hair dryer", "Heating", "Washer"]', bedrooms='2', price='201.00')

In [62]:
q = session.query(Listing)
q = q.filter(Listing.listing_id == 25)
q.update({Listing.price: Listing.price +1})
print(q.first())

Listing(listing_name='Bright & beautiful apartment 10 mins from centre', listing_url='https://www.airbnb.com/rooms/24439562', amenities='["Microwave", "First aid kit", "Refrigerator", "Dishes and silverware", "Oven", "Patio or balcony", "Stove", "Wifi", "Hot water", "Hangers", "Shampoo", "Luggage dropoff allowed", "Kitchen", "Cooking basics", "Iron", "Dishwasher", "Host greets you", "Dedicated workspace", "Paid parking off premises", "Essentials", "Extra pillows and blankets", "Bed linens", "Hair dryer", "Heating", "Washer"]', bedrooms='2', price='207.00')


## Метод delete

In [65]:
q = session.query(Listing)
q = q.filter(Listing.listing_name.like("%Bright & beautiful%"))
to_del = q.one()
session.delete(to_del)
session.commit()
to_del = q.first()
print(to_del)

MultipleResultsFound: Multiple rows were found when exactly one was required

## Join

In [69]:
q = session.query(Listing)
q = q.join(Neighbourhood)
result = q.filter(Neighbourhood.neigh_name == "Centrum-Oost").all()
pprint(result)

[Listing(listing_name='Studio with private bathroom in the centre 1', listing_url='https://www.airbnb.com/rooms/20168', amenities='["Wifi", "Hot water", "Hangers", "Host greets you", "Long term stays allowed", "Carbon monoxide alarm", "Fire extinguisher", "Dedicated workspace", "Paid parking off premises", "Essentials", "Bed linens", "Hair dryer", "TV", "Heating", "Refrigerator", "Free street parking", "Smoke alarm"]', bedrooms='1', price='236.00'),
 Listing(listing_name='COSY HOUSEBOAT IN THE CITY CENTER', listing_url='https://www.airbnb.com/rooms/3086023', amenities='["Wifi", "Carbon monoxide alarm", "Fire extinguisher", "Kitchen", "Cable TV", "TV", "Heating", "Washer", "Smoke alarm"]', bedrooms='1', price='140.00'),
 Listing(listing_name='View on the canals', listing_url='https://www.airbnb.com/rooms/11495829', amenities='["Wifi", "Iron", "Hangers", "Shampoo", "Dedicated workspace", "Dryer", "Essentials", "Kitchen", "Hair dryer", "TV", "Heating", "Indoor fireplace", "Washer"]', bedr

In [70]:
q = session.query(Listing)
q = q.outerjoin(Neighbourhood)
result = q.filter(Neighbourhood.neigh_name == "Centrum-Oost").all()
pprint(result)

[Listing(listing_name='Studio with private bathroom in the centre 1', listing_url='https://www.airbnb.com/rooms/20168', amenities='["Wifi", "Hot water", "Hangers", "Host greets you", "Long term stays allowed", "Carbon monoxide alarm", "Fire extinguisher", "Dedicated workspace", "Paid parking off premises", "Essentials", "Bed linens", "Hair dryer", "TV", "Heating", "Refrigerator", "Free street parking", "Smoke alarm"]', bedrooms='1', price='236.00'),
 Listing(listing_name='COSY HOUSEBOAT IN THE CITY CENTER', listing_url='https://www.airbnb.com/rooms/3086023', amenities='["Wifi", "Carbon monoxide alarm", "Fire extinguisher", "Kitchen", "Cable TV", "TV", "Heating", "Washer", "Smoke alarm"]', bedrooms='1', price='140.00'),
 Listing(listing_name='View on the canals', listing_url='https://www.airbnb.com/rooms/11495829', amenities='["Wifi", "Iron", "Hangers", "Shampoo", "Dedicated workspace", "Dryer", "Essentials", "Kitchen", "Hair dryer", "TV", "Heating", "Indoor fireplace", "Washer"]', bedr

## Группировка

In [73]:
q = session.query(Host.host_name, func.count(Host.host_name)).group_by(Host.host_name)
q.all()

[('A', 2),
 ('Aafke', 1),
 ('Abel', 2),
 ('Achayla', 1),
 ('Ad', 1),
 ('Adam', 3),
 ('Adelle', 1),
 ('Adem', 1),
 ('Adriaan', 1),
 ('Adrian', 1),
 ('Adrien', 2),
 ('Aemilia', 1),
 ('Aernoud', 1),
 ('Afroditi', 1),
 ('Agnes', 1),
 ('Agnesa', 1),
 ('Ahmed', 1),
 ('Aida', 1),
 ('Aiya', 1),
 ('Aj', 1),
 ('Alan', 4),
 ('Albert', 1),
 ('Albertine', 2),
 ('Aldo', 1),
 ('Alec', 1),
 ('Alecsander', 1),
 ('Aleksander', 1),
 ('Aleksandra', 1),
 ('Alessandro', 1),
 ('Alessio', 1),
 ('Alex', 3),
 ('Alexander', 5),
 ('Alexandra', 6),
 ('Alexis', 1),
 ('Alfons', 1),
 ('Alfred', 1),
 ('Alice', 1),
 ('Alina', 1),
 ('Aline', 1),
 ('Altagracia', 1),
 ('Aly And Jeroen', 1),
 ('Alycia', 1),
 ('Amanda', 1),
 ('Amandine', 1),
 ('Amar', 1),
 ('Amber', 3),
 ('Ambra', 1),
 ('Ameen', 1),
 ('Amelie', 1),
 ('Ami', 1),
 ('Amstel BNB & We  Box Homes', 1),
 ('Amstelview Apartment Riverside Plus', 1),
 ('Amy', 3),
 ('Ana', 2),
 ('Ana Leah', 1),
 ('Anand', 1),
 ('Andor', 1),
 ('Andre', 1),
 ('Andrea', 7),
 ('Andreas', 